In [5]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import os
from pydantic import BaseModel

In [12]:
load_dotenv(override=True)

True

In [51]:
openai_api_key = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')



In [8]:
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
llama = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [ ]:
import os
import shutil
import json
from datetime import datetime, timezone, timedelta
from pathlib import Path

In [18]:

STORAGE_ROOT = Path(os.getenv("STORAGE_ROOT")).resolve()
METADATA_FILE = STORAGE_ROOT / os.getenv("METADATA_FILE")




In [ ]:
import os
import json
import shutil
from pathlib import Path
from datetime import datetime
import pytz

# Load from .env or fallback
STORAGE_ROOT = Path(os.getenv("STORAGE_ROOT", "../../storage")).resolve()
METADATA_FILE = STORAGE_ROOT / os.getenv("METADATA_FILE", "file_metadata.json")

# Ensure storage and metadata file exist
STORAGE_ROOT.mkdir(parents=True, exist_ok=True)
if not METADATA_FILE.exists():
    with open(METADATA_FILE, "w") as f:
        json.dump([], f)


def update_last_accessed(files):
    """Update last accessed time for the given metadata entries."""
    metadata = load_metadata()
    updated = False
    for entry in metadata:
        for f in files:
            if entry["path"] == f["path"]:
                entry["last_accessed"] = get_current_time()  # unified format
                updated = True
    if updated:
        save_metadata(metadata)

def load_metadata():
    if METADATA_FILE.exists():
        with open(METADATA_FILE, "r") as f:
            try:
                data = json.load(f)
                return data if isinstance(data, list) else []
            except json.JSONDecodeError:
                return []
    return []

def save_metadata(data):
    with open(METADATA_FILE, "w") as f:
        json.dump(data, f, indent=4)

def get_current_time():
    tz = pytz.timezone("Asia/Kolkata")
    return datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")

def add_file(file_path, category, tag=None, name=None, overwrite=False):
    """
    Uploads a file into STORAGE_ROOT/category/ and records metadata.

    file_path: local path of the uploaded file
    category: folder name inside STORAGE_ROOT
    tag: label to group/search files
    name: optional new name for the file (without extension is fine)
    overwrite: whether to overwrite if a file with same name exists
    """
    src = Path(file_path).resolve()
    if not src.exists():
        raise FileNotFoundError(f"File not found: {src}")

    category_dir = STORAGE_ROOT / category
    category_dir.mkdir(parents=True, exist_ok=True)

    # Use custom name if given, else original name
    final_name = f"{name}{src.suffix}" if name else src.name
    dst = category_dir / final_name

    # Handle duplicates
    if dst.exists() and not overwrite:
        base, ext = dst.stem, dst.suffix
        i = 1
        while True:
            new_name = f"{base}_{i}{ext}"
            new_dst = category_dir / new_name
            if not new_dst.exists():
                dst = new_dst
                final_name = new_name
                break
            i += 1

    shutil.copy2(src, dst)

    # Update metadata
    metadata = load_metadata()
    entry = {
        "name": final_name,
        "path": str(dst),
        "category": category,
        "tag": tag,
        "last_accessed": get_current_time(),  # unified time format
    }
    metadata.append(entry)
    save_metadata(metadata)

    print(f"✅ File uploaded and stored at {dst}")
    return dst



def get_files(name=None, category=None, tag=None):
    """
    Dynamically fetch files based on given filters:
      - name + category + tag -> that exact file
      - name only -> file(s) with that name
      - category only -> all files in that category
      - tag only -> all files with that tag
      - tag + category -> files with that tag in category
      - name + category -> files with that name in category
      - name + tag -> files with that name and tag
    Also updates 'last_accessed' for matched files.
    """
    metadata = load_metadata()
    results = []

    for entry in metadata:
        if name and entry["name"] != name:
            continue
        if category and entry["category"] != category:
            continue
        if tag and entry["tag"] != tag:
            continue
        results.append(entry)

    # update last_accessed timestamp
    if results:
        update_last_accessed(results)

    return results


def delete_files(name=None, category=None, tag=None):
    """
    Deletes files + updates metadata based on filters:
    - category only -> deletes all files in that category
    - tag only -> deletes all files with that tag
    - name only -> deletes that file (wherever it exists)
    - name + category + tag -> deletes that exact file
    - tag + category -> deletes files with that tag in the given category
    """
    metadata = load_metadata()
    updated_metadata = []
    deleted = []

    for entry in metadata:
        match = True

        if name and entry["name"] != name:
            match = False
        if category and entry["category"] != category:
            match = False
        if tag and entry["tag"] != tag:
            match = False

        if match:
            file_path = Path(entry["path"])
            if file_path.exists():
                try:
                    file_path.unlink()  # delete file
                except Exception as e:
                    print(f"⚠️ Could not delete {file_path}: {e}")
            deleted.append(entry)  # remove from metadata
        else:
            updated_metadata.append(entry)

    save_metadata(updated_metadata)

    if deleted:
        print(f"🗑 Deleted {len(deleted)} file(s) and updated metadata:")
        for d in deleted:
            print(f"   - {d['name']} (category={d['category']}, tag={d['tag']})")
    else:
        print("⚠️ No files matched your criteria.")

def get_all_categories_and_tags():
    """
    Return a dict with all unique categories and tags in storage.
    Example:
    {
        "categories": ["guitar_stuff", "work_docs"],
        "tags": ["sample", "important"]
    }
    """
    metadata = load_metadata()
    categories = set()
    tags = set()

    for entry in metadata:
        if "category" in entry and entry["category"]:
            categories.add(entry["category"])
        if "tag" in entry and entry["tag"]:
            tags.add(entry["tag"])

    return {
        "categories": list(categories),
        "tags": list(tags)
    }

